In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [4]:
import mlflow

mlflow.set_tracking_uri("http://18.200.196.114:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlops-bucket-mmjd/2', experiment_id='2', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [10]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [11]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 15.136777093556063


C:\Users\MARCOS\anaconda3\envs\mlops\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [13]:
from mlflow.tracking import MlflowClient


In [ ]:
MLFLOW_TRACKING_URI = 'http://18.200.196.114:5000'
RUN_ID = 'b4d3bca8aa8e46a6b8257fe4541b1136'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [15]:
client.list_experiments()

[<Experiment: artifact_location='s3://mlops-bucket-mmjd/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlops-bucket-mmjd/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlops-bucket-mmjd/2', experiment_id='2', lifecycle_stage='active', name='green-taxi-duration', tags={}>]

In [21]:
run_id = client.list_run_infos(experiment_id='2')[0].run_id
model_name = 'green-taxi-duration-RandomForest'

In [20]:

mlflow.register_model(model_uri=f'runs:/{run_id}/model', name=model_name)

Successfully registered model 'green-taxi-duration-RandomForest'.
2022/06/10 14:30:45 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: green-taxi-duration-RandomForest, version 1
Created version '1' of model 'green-taxi-duration-RandomForest'.


<ModelVersion: creation_timestamp=1654864246836, current_stage='None', description='', last_updated_timestamp=1654864246836, name='green-taxi-duration-RandomForest', run_id='6574d0d7c4b044f585c08de11b3582c4', run_link='', source='s3://mlops-bucket-mmjd/2/6574d0d7c4b044f585c08de11b3582c4/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [23]:
client.transition_model_version_stage(name=model_name, version=1,stage='production')

<ModelVersion: creation_timestamp=1654864246836, current_stage='Production', description='', last_updated_timestamp=1654864620932, name='green-taxi-duration-RandomForest', run_id='6574d0d7c4b044f585c08de11b3582c4', run_link='', source='s3://mlops-bucket-mmjd/2/6574d0d7c4b044f585c08de11b3582c4/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [21]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()